## Tarea 7 del curso CM-072

* Nombre y apellidos: Luis Vasquez Espinoza
* Fecha de presentación: 17 de octubre.

LendingClub es una compañía de préstamos *peer-to-peer* que conecta directamente a los prestatarios con potenciales prestamistas/inversionistas.

Construirás un modelo de clasificación para predecir si un préstamo realizado a través del LendingClub tiene probabilidad de no ser pagado.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

loans = pd.read_csv("lending-club-data.csv", low_memory=False)

1 . Carga en una variable de nombre `todo_columnas` el nombre de todas las columnas del conjunto de datos.

In [3]:
# Solucion
todo_columnas = list(loans)
todo_columnas

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'is_inc_v',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'not_compliant',
 'status',
 'inactive_loans',
 'bad_loans',
 'emp_length_num',
 'grade_num',
 'sub_grade_num',
 'delinq_2yrs_zero',
 'pub_rec

2 . La columna que contiene la información que queremos predecir se llama `malos_prestamos`. En esta columna, el valor 1 significa un préstamo riesgoso (malo), mientras que 0 significa un préstamos seguro.

Para hacer el trabajo más intuitivo, crea una nueva columna `prestamos_seguros` con el siguiente valor:

* +1 si es un préstamo seguro
* -1 si es un préstamos riesgoso (malo)

In [4]:
# Solucion
loans['prestamos_seguros'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)

3 . Calcula la distribución en porcentaje de préstamos malos y préstamos buenos (debe sumar 100%).

In [5]:
# Tu solución
import collections

counter = collections.Counter(loans['prestamos_seguros'])
print(counter)
porcentaje_buenos = counter[1]/len(loans['prestamos_seguros'])
porcentaje_malos = counter[-1]/len(loans['prestamos_seguros'])
print("%P. Buenos: ", porcentaje_buenos)
print("%P. Malos: ", porcentaje_malos)
print("Total: ", porcentaje_malos+porcentaje_buenos)


Counter({1: 99457, -1: 23150})
%P. Buenos:  0.8111853319957262
%P. Malos:  0.18881466800427382
Total:  1.0


4 . Una manera de abordar conjuntos de datos desbalanceados es con un submuestreo  de la clase más grande hasta que la distribución de clases sea mitad y mitad. Vamos a realizar un submuestreo de los préstamos buenos para balancear nuestro conjunto de datos. Ello significa que vamos a descartar muchas observaciones. 

* Pon en una variable `prestamos_arriesgado` todos y solo los préstamos malos.
* Pon en una variable `prestamos_seguros` una muestra aleatoria de préstamos buenos **del mismo tamaño** que la cantidad de préstamos malos. (Usa [pandas.DataFrame.sample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html) con el atributo `random_state=0`)
* Junta en una nueva variable `prestamos_balanceados`, los dos grupos anteriores: `prestamos_arriesgados` y `prestamos_seguros`.

In [6]:
# Tus soluciones
prestamos_arriesgados = loans[loans['prestamos_seguros'] == -1]
prestamos_seguros = loans[loans['prestamos_seguros'] == 1].sample(len(prestamos_arriesgados), random_state=0)
prestamos_balanceados = prestamos_arriesgados.append(prestamos_seguros)
len(prestamos_balanceados)

46300

5 . Asigna a una variable `subconjunto_prestamos` sólo el siguiente subconjunto de características que son las que usaremos:

```python
caracteristica = ['grade',               # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]
```

Asimismo, asigna a una variable **`y`** los valores de la columna `prestamos_seguros`.

In [7]:
# Tu solucion
# Asumiendo que se quiere utilizar el dataset balanceado creado en el ej. anterior
caracteristica = ['grade',               # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

subconjunto_prestamos = prestamos_balanceados.loc[:,caracteristica]
y = prestamos_balanceados['prestamos_seguros']

6 .  Usando [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) convierte las variables categóricas de `subconjunto_prestamos` en variables numéricas *one-hot*. Guarda el nuevo conjunto de datos en `X`.

In [8]:
# Tu solucion
X = pd.get_dummies(subconjunto_prestamos)
X.head()

,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,grade_A,grade_B,grade_C,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
1,1,1,1.00,1,1,9.4,0.0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
6,0,5,5.55,1,1,32.6,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
7,1,1,18.08,1,1,36.5,0.0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
10,1,1,10.08,1,1,91.7,0.0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
12,0,4,7.06,1,1,55.5,0.0,0,1,0,...,0,0,0,0,1,0,0,0,1,0


7 .  Empleando `sklearn.model_selection.train_test_split` separa el conjunto de datos en un 90% para entrenamiento y validación (`X_entrenamiento_val`, `y_entrenamiento_val`), y 10% para pruebas (`X_prueba`, `y_prueba`).

Luego separa (`X_entrenamiento_val`, `y_entrenamiento_val`) en un 80% para entrenamiento (`X_entrenamiento`, `y_entrenamiento`) y 20% para validación (`X_val`, `y_val`).

In [10]:
# Tu solucion
from sklearn.model_selection import train_test_split
# Particion 90,10
X_entrenamiento_val, X_prueba, y_entrenamiento_val, y_prueba = train_test_split(X, y, test_size=0.1) 
X_entrenamiento, X_val, y_entrenamiento, y_val = train_test_split(X_entrenamiento_val, y_entrenamiento_val, test_size=0.2)

8 . Entrena un modelo como Regresión Logística, Naive Bayes, KNN y un cuarto modelo de tu elección, con las siguientes indicaciones:

* Utilizar el uso apropiado de la normalización (Scaling) de datos si fuese necesario.
* El uso apropiado de una técnica para la selección de los mejores parámetros de cada modelo (p.ej. búsqueda grid o búsqueda aleatoria)
* Reporte para cada modelo la exactitud , precisión y exhaustividad, F1-Score  **en el conjunto de pruebas.** y muestra la matriz de confusión.
* Comenta tus resultados.

In [73]:
from sklearn import metrics as m

def showMetrics(y, y_predicted, f1_flag=False):
    print("Total: ", len(y))
    print("Exactitud no normalizada:", m.accuracy_score(y, y_predicted, normalize=False))
    print("Exactitud normalizada (no normaliz./total):",m.accuracy_score(y, y_predicted, normalize=False)/len(y))
    print("Presicion:", m.precision_score(y, y_predicted))
    print("Exhaustividad:", m.recall_score(y, y_predicted))
    if f1_flag:
        print("F1-Score:", m.f1_score(y, y_predicted))
    matriz_confusion = m.confusion_matrix(y, y_predicted)
    print("Matriz de conf:\n", matriz_confusion)
    
def showAllMetrics(X_train, X_test, X_val, y_train, y_test, y_val, model):
    for text, y_real, current_X in zip(["PRUEBA", "ENTRENAMIENTO", "VALIDACION"],
                                       [y_test, y_train, y_val],
                                       [X_test, X_train, X_val]
                                      ):
        print(text)
        print("_____________")
        showMetrics(y_real, model.predict(current_X), True if text=="PRUEBA" else False)
        print("=========================================")


In [74]:
# Tus soluciones
# A. Usando regresion logistica
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

lr = LogisticRegression()
lr.fit(X_entrenamiento, y_entrenamiento)
showAllMetrics(X_entrenamiento, X_prueba, X_val, y_entrenamiento, y_prueba, y_val, lr)
print("< SELECCCIONANDO FEATURES VIA RFE>")
lr = RFE(lr, 3) # Seleccion de Features usando Recursive Feature Elimination
lr.fit(X_entrenamiento, y_entrenamiento)
showAllMetrics(X_entrenamiento, X_prueba, X_val, y_entrenamiento, y_prueba, y_val, lr)

PRUEBA
_____________
Total:  4630
Exactitud no normalizada: 2954
Exactitud normalizada (no normaliz./total): 0.6380129589632829
Presicion: 0.6430071398572028
Exhaustividad: 0.649554518455664
F1-Score: 0.6462642465175179
Matriz de conf:
 [[1423  850]
 [ 826 1531]]
ENTRENAMIENTO
_____________
Total:  33336
Exactitud no normalizada: 21577
Exactitud normalizada (no normaliz./total): 0.6472582193424526
Presicion: 0.6476293752635701
Exhaustividad: 0.6452581032412965
Matriz de conf:
 [[10827  5849]
 [ 5910 10750]]
VALIDACION
_____________
Total:  8334
Exactitud no normalizada: 5311
Exactitud normalizada (no normaliz./total): 0.6372690184785217
Presicion: 0.6341876208897486
Exhaustividad: 0.6346479554802806
Matriz de conf:
 [[2688 1513]
 [1510 2623]]
< SELECCCIONANDO FEATURES VIA RFE>
PRUEBA
_____________
Total:  4630
Exactitud no normalizada: 2649
Exactitud normalizada (no normaliz./total): 0.5721382289416846
Presicion: 0.770893371757925
Exhaustividad: 0.22698345354263894
F1-Score: 0.35070468

## Comentarios (Logistic Regression)

Se notan scores similares con valores alrededor de 0.64. Posteriormente, habiendo aplicado la selección de features se nota un aumento considerable en la presición de las estimaciones para todos los bloques de datos *train_test_val*, aumentando hasta en un 10%; sin embargo la exactitud decae en un 10%, el F1-Score (ver modulos de PRUEBA) en un 30% y la exhaustividad en un alarmante 40%; esto debido a la eliminación de muchas características necesarias para la correcta clasificación. Considero que es mejor aplicar este modelo a la mayor cantidad de *features* disponibles o almenos sesgar la menor cantidad de columnas de la data original. El tiempo de entrenamiento optimizable al usar RFE no se distancia mucho del modelo original, por lo que optimizar el proceso tampoco es una buena razón para usarlo.

In [77]:
# B. Usando Naive Bayes
from sklearn.naive_bayes import MultinomialNB

m_naive_bayes = MultinomialNB()
m_naive_bayes.fit(X_entrenamiento, y_entrenamiento)
showAllMetrics(X_entrenamiento, X_prueba, X_val, y_entrenamiento, y_prueba, y_val, m_naive_bayes)
print("< SELECCCIONANDO FEATURES VIA RFE>")
m_naive_bayes = RFE(m_naive_bayes, 10) # Seleccion de Features usando Recursive Feature Elimination
m_naive_bayes.fit(X_entrenamiento, y_entrenamiento)
showAllMetrics(X_entrenamiento, X_prueba, X_val, y_entrenamiento, y_prueba, y_val, m_naive_bayes)

PRUEBA
_____________
Total:  4630
Exactitud no normalizada: 2741
Exactitud normalizada (no normaliz./total): 0.5920086393088553
Presicion: 0.5618393234672304
Exhaustividad: 0.9019940602460755
F1-Score: 0.6923953753460348
Matriz de conf:
 [[ 615 1658]
 [ 231 2126]]
ENTRENAMIENTO
_____________
Total:  33336
Exactitud no normalizada: 19650
Exactitud normalizada (no normaliz./total): 0.5894528437724982
Presicion: 0.5554436987322894
Exhaustividad: 0.8941776710684274
Matriz de conf:
 [[ 4753 11923]
 [ 1763 14897]]
VALIDACION
_____________
Total:  8334
Exactitud no normalizada: 4862
Exactitud normalizada (no normaliz./total): 0.5833933285337173
Presicion: 0.5492475040977499
Exhaustividad: 0.8918461166223083
Matriz de conf:
 [[1176 3025]
 [ 447 3686]]
< SELECCCIONANDO FEATURES VIA RFE>
PRUEBA
_____________
Total:  4630
Exactitud no normalizada: 2259
Exactitud normalizada (no normaliz./total): 0.4879049676025918
Presicion: 0.36538461538461536
Exhaustividad: 0.008061094611794654
F1-Score: 0.0157

## Comentarios (Naive Bayes)
Inicialmente se observan scores aceptables, con exhaustividades bastante altas que evidencian *pocos falsos negativos*, por lo que el modelo al menos ahorra en no clasificar un dato como "mal préstamo" pudiendo evitar la mitad de la catástrofe financiera en favor al cliente; sin embargo la presición es muy baja, lo que podría tener como consecuencia habilitar préstamos que le cuesten más a la empresa *(Lending Club)* por no prever estos casos. Pero la tendencia de la clasificación anterior se mantiene: ni bien consideramos menos *features* de la data notamos un bajón de scores en todos los campos, especialmente para este modelo. Si bien la exactitud no se ven tan afectada (~-3%), la presición y exhaustividad evidencian cómo el modelo se vuelve directamente inútil. Mi conclusión para este caso: más (propiedades) es mejor.

In [82]:
# C. Usando KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X, y)
showAllMetrics(X_entrenamiento, X_prueba, X_val, y_entrenamiento, y_prueba, y_val, knn)

PRUEBA
_____________
Total:  4630
Exactitud no normalizada: 3573
Exactitud normalizada (no normaliz./total): 0.7717062634989201
Presicion: 0.7692626346313173
Exhaustividad: 0.7878659312685617
F1-Score: 0.7784531544749529
Matriz de conf:
 [[1716  557]
 [ 500 1857]]
ENTRENAMIENTO
_____________
Total:  33336
Exactitud no normalizada: 25884
Exactitud normalizada (no normaliz./total): 0.7764578833693304
Presicion: 0.7696497598688063
Exhaustividad: 0.7887755102040817
Matriz de conf:
 [[12743  3933]
 [ 3519 13141]]
VALIDACION
_____________
Total:  8334
Exactitud no normalizada: 6438
Exactitud normalizada (no normaliz./total): 0.7724982001439885
Presicion: 0.7644833293922914
Exhaustividad: 0.7822405032663925
Matriz de conf:
 [[3205  996]
 [ 900 3233]]


## Comentarios (KNN)
No me fue posible probar la selección de *features* para knn por el método recursivo habitual. Los scores obtenidos son muy buenos en todos los aspectos; no evidencian *overfitting* y dan la suficiente confianza como para aplicarse. Supongo que al ser KNN el único downfall es el tiempo de ejecución del modelo.

In [89]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=5)
dtc.fit(X, y)
showAllMetrics(X_entrenamiento, X_prueba, X_val, y_entrenamiento, y_prueba, y_val, dtc)
print("< SELECCCIONANDO FEATURES VIA RFE>")
dtc = RFE(dtc, 35) # Seleccion de Features usando Recursive Feature Elimination
dtc.fit(X_entrenamiento, y_entrenamiento)
showAllMetrics(X_entrenamiento, X_prueba, X_val, y_entrenamiento, y_prueba, y_val, dtc)

PRUEBA
_____________
Total:  4630
Exactitud no normalizada: 2931
Exactitud normalizada (no normaliz./total): 0.6330453563714903
Presicion: 0.6210448859455482
Exhaustividad: 0.7161646160373356
F1-Score: 0.6652216748768472
Matriz de conf:
 [[1243 1030]
 [ 669 1688]]
ENTRENAMIENTO
_____________
Total:  33336
Exactitud no normalizada: 21127
Exactitud normalizada (no normaliz./total): 0.6337592992560596
Presicion: 0.6171500763278412
Exhaustividad: 0.7037214885954381
Matriz de conf:
 [[ 9403  7273]
 [ 4936 11724]]
VALIDACION
_____________
Total:  8334
Exactitud no normalizada: 5196
Exactitud normalizada (no normaliz./total): 0.6234701223902088
Presicion: 0.6056487576980251
Exhaustividad: 0.6900556496491652
Matriz de conf:
 [[2344 1857]
 [1281 2852]]
< SELECCCIONANDO FEATURES VIA RFE>
PRUEBA
_____________
Total:  4630
Exactitud no normalizada: 2923
Exactitud normalizada (no normaliz./total): 0.631317494600432
Presicion: 0.625
Exhaustividad: 0.6894357233771744
F1-Score: 0.6556384910227961
Matr

## Comentarios (DecisionTreeClassifier)
Para mi sorpresa este modelo es el único que soporta la eliminación de características sin sufrir tanto en las métricas y optimizando el proceso de entrenamiento. Se notan valores similares antes y despues del sesgado y en ambos casos se producen scores más aceptables que el segundo o incluso el primer modelo. Si se necesita correr el modelo en sistemas low-end como embebidos esta es la mejor alternativa (si no se añade en consideración el tiempo de RFE).